In [28]:
import os 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [25]:
# 현재 작업 디렉터리 확인 및 파일 목록 보기
print("Current working directory:", os.getcwd())
print("Files in the current directory:", os.listdir())

Current working directory: g:\다른 컴퓨터\Desktop\학교자료\학교 자료\.학사과정\4학년1학기\비즈니스 어낼리틱스\Code\BOD_Prediction\conformal_prediction
Files in the current directory: ['predict.ipynb']


In [26]:
# CSV 파일 불러오기
bod_original = pd.read_csv('../../dataset/Original Dataset.csv')
bod_entire = pd.read_csv('../../dataset/Entire_dataset_5days_ago_final.csv')

# 데이터프레임의 첫 5개 행 출력
print("Original Dataset first 5 rows:")
print(bod_original.head())
print("Entire Dataset first 5 rows:")
print(bod_entire.head())

Original Dataset first 5 rows:
   location  company  reg_date  month  temper  rainfall  water_temper  in_bod  \
0        92    87103  20190101      1    -4.8       0.0           6.6    18.9   
1        92    87103  20190102      1    -4.4       0.0           6.6    16.9   
2        92    87103  20190103      1    -5.5       0.0           6.6    21.1   
3        92    87103  20190104      1    -2.8       0.0           6.5    16.4   
4        92    87103  20190105      1    -1.3       0.0           6.5    17.4   

   in_cod  in_ss   in_tn  in_tp  in_bac   flux  
0    21.0   13.3  14.184  0.946  5000.0  413.0  
1    19.0   10.0  13.651  0.955  5000.0  357.0  
2    18.4   17.3  17.357  0.850  4000.0  414.0  
3    18.3   10.3  19.003  0.886  2000.0  400.0  
4    18.2   11.7  18.773  0.907  4000.0  353.3  
Entire Dataset first 5 rows:
   location  company    reg_date  temper  rainfall  water_temper  in_bod  \
0        92    87103  2019-01-06    -5.5       0.0           6.6    16.2   
1      

# 데이터 분포 확인

In [27]:
# 1) 변수 타입별로 분류
numeric_cols     = bod_original.select_dtypes(include=['int64','float64']).columns.tolist()
categorical_cols = bod_original.select_dtypes(include=['object','category']).columns.tolist()

# 2) 수치형 변수 분포 확인 (히스토그램 + KDE)
for col in numeric_cols:
    plt.figure(figsize=(6,4))
    sns.histplot(bod_original[col].dropna(), kde=True, bins=30)
    plt.title(f'{col} Distribution (Histogram + KDE)')
    plt.xlabel(col)
    plt.ylabel('빈도')
    plt.tight_layout()
    plt.show()

# 3) 범주형 변수 분포 확인 (막대 그래프)
for col in categorical_cols:
    plt.figure(figsize=(6,4))
    # 만약 고유값이 너무 많다면 상위 N개만 표시
    order = bod_original[col].value_counts().index
    sns.countplot(data=bod_original, y=col, order=order)
    plt.title(f'{col} Countplot (Countplot)')
    plt.xlabel('num')
    plt.ylabel(col)
    plt.tight_layout()
    plt.show()

ModuleNotFoundError: No module named 'matplotlib.backends._backend_agg'

In [ ]:
# 상관관계 분석
from scipy.stats import pearsonr

# 2) 수치형 변수만 선택하고 in_bod는 제외
numeric_cols = bod_original.select_dtypes(include=['int64', 'float64']).columns.tolist()
numeric_cols.remove('in_bod')

# 3) 각 변수와 in_bod 간의 Pearson 상관계수 및 p-value 계산
results = []
for col in numeric_cols:
    clean_df = bod_original[['in_bod', col]].dropna()
    r, p = pearsonr(clean_df['in_bod'], clean_df[col])
    results.append({'feature': col, 'pearson_r': r, 'p_value': p})

# 4) 결과 DataFrame 생성 및 p-value 기준 정렬
results_df = pd.DataFrame(results).sort_values('p_value').reset_index(drop=True)

# 5) 결과 출력
print(results_df)

         feature  pearson_r        p_value
0         in_cod   0.756993   0.000000e+00
1          in_ss   0.630978   0.000000e+00
2          in_tp   0.642264   0.000000e+00
3          in_tn   0.626477   0.000000e+00
4         in_bac   0.375305  2.436249e-244
5           flux  -0.250022  4.702577e-105
6        company  -0.221365   3.389583e-82
7       rainfall  -0.075199   1.104565e-10
8         temper  -0.073405   3.023375e-10
9   water_temper   0.026924   2.104407e-02
10      reg_date   0.019017   1.032171e-01
11         month   0.008526   4.650992e-01
12      location        NaN            NaN


C:\Users\vprai\AppData\Local\Temp\ipykernel_39984\1409678882.py:12: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  r, p = pearsonr(clean_df['in_bod'], clean_df[col])


| 순위 | 변수            | 상관계수(r) | p-값      | 해석                                        |
| -- | ------------- | ------- | -------- | ----------------------------------------- |
| 1  | in\_cod       | +0.757  | ≈0       | BOD와 가장 강한 양의 선형관계. COD가 높을수록 BOD도 높아짐.   |
| 2  | in\_tp        | +0.642  | ≈0       | 총인과도 강한 양의 상관. 영양염류 농도가 높을수록 BOD 상승 경향.   |
| 3  | in\_ss        | +0.631  | ≈0       | 부유물질(SS)도 뚜렷한 양의 상관을 보임.                  |
| 4  | in\_tn        | +0.626  | ≈0       | 총질소 역시 높은 양의 상관관계.                        |
| 5  | in\_bac       | +0.375  | 2.4e-244 | 대장균군수와 중간 정도 양의 상관.                       |
| 6  | flux          | –0.250  | 4.7e-105 | 유량과는 음의 상관. 유량이 많을수록 BOD가 희석되어 낮아지는 경향.   |
| 7  | company       | –0.221  | 3.4e-82  | 회사 ID와 상관이 있지만, 수치형 인코딩된 범주형 변수라 해석 유의.   |
| 8  | rainfall      | –0.075  | 1.1e-10  | 강우량과 약한 음의 상관. 비가 올 때 BOD가 소폭 낮아질 수 있음.   |
| 9  | temper        | –0.073  | 3.0e-10  | 기온과도 약한 음의 상관. 낮은 기온일수록 BOD가 소폭 높아질 수 있음. |
| 10 | water\_temper | +0.027  | 0.021    | 수온과 거의 무시해도 될 수준의 약한 양의 상관.               |
| 11 | reg\_date     | +0.019  | 0.103    | 날짜 자체(시간 경과)와는 유의미한 상관 없음.                |
| 12 | month         | +0.009  | 0.465    | 월별 계절성 지표도 BOD와 관계 거의 없음.                 |


In [ ]:
# 2) 수치형 변수만 선택
numeric_cols = bod_original.select_dtypes(include=['int64', 'float64']).columns.tolist()

# 3) 상관계수 행렬 계산
corr_matrix = bod_original[numeric_cols].corr()

# 4) 히트맵 그리기
plt.figure(figsize=(10, 8))
plt.imshow(corr_matrix, aspect='auto')
plt.colorbar(label='Correlation Coefficient')
plt.xticks(range(len(numeric_cols)), numeric_cols, rotation=90)
plt.yticks(range(len(numeric_cols)), numeric_cols)
plt.title('Numeric Features Correlation Matrix Heatmap')
plt.tight_layout()
plt.show()


NameError: name 'bod_original' is not defined

# SARIMAX 모델을 사용하여 예측하기 위한 데이터 전처리

In [ ]:
#SARIMAX 모델을 사용하여 예측하기 위한 데이터 전처리